# Graphical Models

###### COMP4670/8600 - Statistical Machine Learning - Tutorial

### Assumed knowledge
- Undirected graphical models (lectures)

### After this lab, you should be comfortable with:
- Basic operations and definitions of graphical models

Setting up the environment

In [1]:
import numpy as np
from IPython.display import display, Image

In the second part of the tutorial, we will talk about Markov random fields (MRFs).

## Markov Random Fields

Markov random fields (MRFs) are similar to Bayesian networks (BNs), but with undirected edges. This means that the causal relationships in BNs are no longer defined in MRFs. The relationship between nodes connected by an edge is only "correlational" now.

Consider an undirected grapical model. Let $C$ denote a *clique* of the graph, corresponding to the set of variables $\mathsf{x}_C$. We assign a *potential function* $\psi_C(\mathsf{x}_C) \geq 0$ to each set. The joint distribution of all variables in that graph is 
$$
p(\mathsf{x}) = \frac{1}{Z} \prod_{C} \psi_C(\mathsf{x}_C),
$$
where $Z = \sum_{x}\prod_{C} \psi_C(\mathsf{x}_C)$ is the *normalising constant* to ensure $p(\mathsf{x})$ sums to 1.

## Converting a Bayesian net to an MRF

To convert a BN into an MRF, we perform the following steps:
1. "Marry" the parents: add additional undirected edges between all pairs of parents for each node in the graph. This step is also called "moralisation".
2. Drop the arrows on all original edges. You should now have an fully undirected graph often called a *moral graph*.
3. For each clique in the undirected graph, intialise its potential to 1.
4. For each conditional probability in the original graph, multiply it into a corresponding clique potential.
5. The normalising constant is simply $Z = 1$.

Note that the resulting MRF may represent different conditional independence statements than the original BN.

Let's work on one example.

**Exercise 1.** Revisit the Bayesian network we worked on last week.

In [2]:
Image(url="https://machlearn.gitlab.io/sml2021/tutorials/graphical_model.png")

Convert this graph into a moral graph. This corresonds to Steps 1 and 2 above. Draw the resulting network.

In [3]:
# Solution 
Image(url="https://machlearn.gitlab.io/sml2021/tutorials/MRF.png")

**Exercise 2.** Answer the following:

(a) Identify the maximal cliques in the graph and write down the corresponding clique potentials. This corresponds to Steps 3 and 4 above.

(b) Then write out the joint distribution of the undirected graph. 

(c) Compare the conditional independence statements of the MRF with the BN.

### Solution

First, the joint distribution of the original directed graph is
$$
p(A, B, C, D, E) = p(B) p(C) p(A \mid B) p(D \mid B, C) p(E \mid D).
$$

(a) 

There are three maximal cliques in this graph: $\{B, C, D\}$, $\{A, B\}$ and $\{E, D\}$. We assign the potentials to these cliques as follows:

- $\psi_1(B,C,D)=p(B)p(C)p(D \mid B,C)$

- $\psi_2(A,B)=p(A\mid B)$

- $\psi_3(E,D)=p(E\mid D)$

(Note that other assignments are possible. For example, since both cliques $\{B, C, D\}$ and $\{A, B\}$ contain node $B$, it is possible to put $p(B)$ within $\psi_1(B,C,D)$ or $\psi_2(A,B)$. Similar for node $D$.)

(b) 

Therefore, the joint distribution of the undirected graph is
$$
p(A, B, C, D, E) = \psi_1(B,C,D) \psi_2(A,B) \psi_3(E,D).
$$
Note that the normalising constant $Z$ is just 1, because we know that the product of conditional probabilities above is a probability, meaning it sums to 1.

(c) 

In a directed graph, we learned to used D-separation to prove/disprove conditional independence. For example, we have shown that $A \perp\!\!\!\perp D \mid B$, because the only (undirected) path from $A$ to $B$ is $A - D - B$, and $D$ is a TT node that is observed, which makes this path blocked.

In an MRF, two nodes are conditionally independent if all paths between them are blocked by observed variables. It is still possible to infer $A \perp\!\!\!\perp D \mid B$, because both paths from $A$ to $B$ contain node $D$, which is observed.

However, some conditional independencies cannot be retained. For example, we have shown that in the directed graph, $B \perp\!\!\!\perp C \mid \emptyset$. This can no longer be inferred from the undirected graph, because $B$ and $C$ are connected by an edge, so they must be in a same clique.

# Sum-Product Algorithm

The aim of this exercise is to implement the sum product algorithm on a chain graph.

## Factor graphs

Revise the definition of a factor graph in part 3 of the lectures (or Section 8.4.3 of the textbook). A nice property about factor graphs is that the joint distribution can can be expressed as a product of factors. This is important later when we revisit the sum-product algorithm.

Here we remind you of how to convert a graph (directed or undirected) into a factor graph.

To convert a directed graph into a factor graph:
1. Add a factor node corresponding to each conditional probability.
2. Assign a conditional probability to the value of its corresponding factor.
3. Connect a factor to its corresponding nodes in the conditional probability. 

To convert an undirected graph into a factor graph:
1. Add a factor node corresponding to each maximal clique.
2. Create a factor for $Z$, which is over an empty set of variables.
3. Assign a clique potential to the value of its corresponding factor.
4. Connect a factor to its corersponding nodes in the original clique.

## Distributive law

The [distributive property](http://en.wikipedia.org/wiki/Distributive_property) can be used to save computation, and is the basis of message passing and dynamic programming. See an [anecdote](http://bibiserv.techfak.uni-bielefeld.de/dynprog/node3_mn.html) about camels.

**Exercise 3.** Consider the following equation:
$$
2*3 + 2*5 = 2 * (3+5).
$$

* How many mathematical operations (multiplications and additions) are on the left hand side?
* How many mathematical operations are on the right hand side?

Construct a larger example where there is even more computational savings.

### Solution

* LHS has 2 multiplies and 1 addition
* RHS has 1 multiply and 1 addition

## Message passing

Consider the following factor graph. 

In [4]:
Image(url="https://machlearn.gitlab.io/sml2021/tutorials/message_passing.png")

The factors are given by the following tables:

|f(A,B)  | A=$\square$ | A=$\bigcirc$ | A = $\clubsuit$ | A = $\heartsuit$ | A = $\triangle$ |
|--|:--:|:--:|:--:|:--:|:--:|
|**B**=$p$|0.01|0.01|0.12|0.01|0.14|
|**B**=$q$|0.03|0.15|0.01|0.01|0.01|
|**B**=$r$|0.13|0.11|0.07|0.18|0.01|

|g(B,C) | B=$p$ | B=$q$ | B=$r$ |
|--|:--:|:--:|:--:|
|**C**=$w$|0.05|0.06|0.07|
|**C**=$x$|0.1|0.3|0.2|
|**C**=$y$|0.03|0.02|0.1|
|**C**=$z$|0.11|0.15|0.08|

|  | h(C) |
|--|:--:|
|**C**=$w$|1.2|
|**C**=$x$|3.2|
|**C**=$y$|1.8|
|**C**=$z$|2.3|

Using the sum product algorithm, compute the marginal distribution of the random variable $B$.

*Hint: Note that the factors are not normalised.*

### Solution

The joint distribution according to this factor graph is
$$
p(A, B, C) = f(A, B) \cdot g(B, C) \cdot h(C).
$$
*(Actually, we must use $\propto$ instead of $=$ because the factors aren't normalised to sum to 1. However, we can just normalise the final result to have a probability, so this won't be a problem. It's only a slight abuse of notation.)*

**1. Determining which messages to compute**

The inference problem requires us to find $p(B)$. First, we identify that there are two neighbouring factor nodes of $B$, which are $f(A, B)$ and $g(B, C)$. Therefore, the marginal probability of $B$ is the product of two messages from $f(A, B)$ and $g(B, C)$, written as
$$
p(B) = \mu_{f(A, B) \rightarrow B} \cdot \mu_{g(B, C) \rightarrow B}.
$$
*(Again, we abuse notation by replacing $\propto$ with $=$ because these messages may not be normalised. This can be easily fixed by normalising $p(B)$ at the end.)*

Now let's call $\mu_{f(A, B) \rightarrow B}$ the *forward direction* message (left to right) and $\mu_{g(B, C) \rightarrow B}$ the *backward direction* message (right to left). We will compute them separately and multiply them together to get $p(B)$.

The message from variable node A
$\mu_{A\to f(A,B)} = 1$

**2. Forward direction** $\mu_{f(A, B) \rightarrow B}$

To get to $B$ from the left, we need to do two steps:
- Go from $A \rightarrow f(A, B)$. This is a message from from a variable leaf node to a factor node, so by (8.70), we have $\mu_{A \rightarrow f(A, B)} = 1$.
- Go from $f(A, B) \rightarrow B$. This is a message from a factor (non-leaf) node to a variable node, so it is:
\begin{align*}
\mu_{f(A,B) \to B} &= \sum_{A} f(A,B) \cdot \mu_{A \rightarrow f(A, B)} = \sum_{A} f(A,B) =
\begin{bmatrix}
0.29\\
0.21\\
0.5\\
\end{bmatrix}.
\end{align*}

**3. Backward direction** $\mu_{g(B, C) \rightarrow B}$

To get to $B$ from the right, we need to do three steps:
- Go from $h(C) \rightarrow C$. This is a message from a factor leaf node to a variable node, so by (8.71) we have $\mu_{h(C) \rightarrow C} = h(C).$
- Go from $C \rightarrow g(B, C).$ This is a message from variable (non-leaf) nodes to a factor node, so it's simply the product of all messages to those variable nodes. There's only one such message, which is $\mu_{h(C) \rightarrow C}$. Therefore, $\mu_{C \rightarrow g(B, C)} = \mu_{h(C) \rightarrow C} = h(C)$.
- Go from $g(B, C) \rightarrow B$. This is a message from a factor (non-leaf) node to a variable node, so it is:
\begin{align*}
\mu_{g(B,C) \to B} = \sum_{C} g(B,C) \cdot \mu_{C \rightarrow g(B, C)} = \sum_{C} g(B,C) \cdot h(C) =
\begin{bmatrix}
0.687\\
1.413\\
1.088\\
\end{bmatrix}.
\end{align*}

**4. Combine messages from both direction**

Therefore, the (unnormalised) marginal probability $p(B)$ is
$
p(B) = \mu_{f(A,B) \to B} \cdot \mu_{g(B,C) \to B}
=
\begin{bmatrix}
0.19923\\
0.29673\\
0.544\\
\end{bmatrix}.
$

To ensure this is actually a probability, we need to normalise it. The final result is
$p(B) = 
\begin{bmatrix}
0.192\\
0.285\\
0.523\\
\end{bmatrix}.
$

### Textbook Questions

- Q8.20: Induction on graph structure (recall from MATH1005/6005) (Difficulty $\star$)
- Q8.21: Note typo: it should be $f_s(x_s)$ (Difficulty $\star\star$)
- Q8.27: Construct example showing greedy method not working (Difficulty $\star\star$)
- Q8.29: Induction on tree structure (recall from MATH1005/6005) (Difficulty $\star\star$)
- Extra: Derive eq 8.74 to 8.85 w.r.t Fig 8.51

- Q10.2: Solving simulataneous equations (Difficulty $\star$)
- Q10.3: Use lagrangian to enforce normalisation of q (Difficulty $\star\star$)
- Q10.6: Hint, how to introduce log term for both p and q? (Difficulty $\star\star$)

- Q2.44: Manipulation with a more complex conjugate to derive the posterior (Difficulty $\star\star$)

- Q10.7: Rewritting to the form of the respective distributions. Mostly algebra. (Difficulty $\star\star$)
- Q10.8: What will $b_n$ be approximated as? (Difficulty $\star$)
- Q10.9: Essentially, deriving 10.31 and 10.32 (Difficulty $\star\star$)